In [1]:
# -*- coding: utf-8 -*-
# %%
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %% [markdown]
# ### Install packages and import
# %%
# #################################### PLEASE INSTALL LATEST CHROME WEBDRIVER #####################################
# Uncomment to run as required
# #     --install-option="--chromedriver-version= *.**" \
#   --install-option="--chromedriver-checksums=4fecc99b066cb1a346035bf022607104,058cd8b7b4b9688507701b5e648fd821"
# %%
# ##### COPY THE LINES IN THIS COMMENT TO THE TOP OF NEW SCRIPTS #####
# # Function to import this package to other files
# import os
# import sys
# from pathlib import Path

# code_dir = None
# code_dir_name = 'Code'
# unwanted_subdir_name = 'Analysis'

# for _ in range(5):

#     parent_path = str(Path.cwd().parents[_]).split('/')[-1]

#     if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

#         code_dir = str(Path.cwd().parents[_])

#         if code_dir is not None:
#             breakconda env export >

# main_dir = str(Path(code_dir).parents[0])
# scraped_data = f'{code_dir}/scraped_data'
# sys.path.append(code_dir)

# from setup_module.imports import *
# from setup_module.params import *
# from setup_module.scraping import *
# from setup_module.classification import *
# from setup_module.vectorizers_classifiers import *

# warnings.filterwarnings("ignore", category=DeprecationWarning)
# %matplotlib notebook
# %matplotlib inline

# %%
import os
import sys
from pathlib import Path

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

main_dir = str(Path(code_dir).parents[0])
scraped_data = f'{code_dir}/scraped_data'
sys.path.append(code_dir)

from setup_module.imports import *
from setup_module.params import *
from setup_module.scraping import *
from setup_module.classification import *
from setup_module.vectorizers_classifiers import *

warnings.filterwarnings("ignore")
%matplotlib notebook
%matplotlib widget
%matplotlib inline

0it [00:00, ?it/s]

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-09 12:10:40.225801: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-09 12:10:40.226070: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Complete Dataset

In [2]:
if classification_final_from_post_cleanup is True and is_non_zero_file(df_dir + f'df_jobs_post_cleanup.{file_save_format}') is False:

    print('Post Cleanup Started.')
    df_jobs = post_cleanup(
        keywords_from_list=True,
        site_from_list=True)
    print('Post Cleanup Finished.')

    # with open(df_dir + f'df_jobs_post_cleanup.{file_save_format_backup}', 'w', newline="") as f:
    #     writer = csv.writer(f)
    #     writer.writerows(df_jobs)

    with open(df_dir + f'df_jobs_post_cleanup.{file_save_format}', 'wb') as f:
        pickle.dump(df_jobs, f)

elif classification_final_from_post_cleanup is False or is_non_zero_file(df_dir + f'df_jobs_post_cleanup.{file_save_format}') is True:

    with open(df_dir + f'df_jobs_post_cleanup.{file_save_format}', 'rb') as f:
        df_jobs = pickle.load(f)

if classification_final_from_df_jobs_list is True:
    print('Reading Jobs List.')
    df_all = []
    for jobs_list in tqdm.tqdm(df_jobs):
        if isinstance(jobs_list, list):
            df_all.extend(d for d in jobs_list if isinstance(d, pd.DataFrame))
        elif isinstance(jobs_list, pd.DataFrame):
            df_all.append(jobs_list)

    print('Concatenating Jobs List.')
    df_all = pd.concat(df_all)

    print('DF from list:')
    df_all.info()
    df_all['Gender'].value_counts()
    df_all['Age'].value_counts()

    df_all.to_pickle(
        f'{df_dir}df_from_list_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format}')

    df_all.to_csv(
        f'{df_dir}df_from_list_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format_backup}',
        index=False,
    )

    df_all.drop_duplicates(
        subset=[
            'Search Keyword',
            'Platform',
            'Job ID',
            'Job Title',
            'Company Name',
            'Job Description',
        ],
        keep='first',
        inplace=True,
        ignore_index=True,
    )

    # Fix Date
    df_all['Collection Date'] = pd.to_datetime(df_all['Collection Date'])

    # Arrange Categories
    df_all = categorize_df_gender_age(df_all)

    # Gender Recode
    df_all = dummy_code_df_gender_age(df_all)

    print('DF Main:')
    df_all.info()
    df_all['Gender'].value_counts()
    df_all['Age'].value_counts()

    df_all.to_pickle(
        f'{df_dir}df_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format}')

    df_all.to_csv(
        f'{df_dir}df_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format_backup}',
        index=False,
    )

    print(df_all['Collection Date'].drop_duplicates().nlargest(5))

    df_all = df_all.loc[df_all['Collection Date'] < '2021-05-01']

    print('DF Fixed Date:')
    df_all.info()
    df_all['Gender'].value_counts()
    df_all['Age'].value_counts()

    df_all.to_pickle(
        f'{df_dir}df_fixed_date_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format}')

    df_all.to_csv(
        f'{df_dir}df_fixed_date_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format_backup}',
        index=False,
    )

    print('Spliting into Sentences.')
    df_all = split_df_jobs_to_df_sent(df_all)

    print('DF Split:')
    df_all.info()
    df_all['Gender'].value_counts()
    df_all['Age'].value_counts()

    df_all.to_pickle(
        f'{df_dir}df_split_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format}')

    df_all.to_csv(
        f'{df_dir}df_split_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format_backup}',
        index=False,
    )

    print('Simple Preprocessing')
    df_all = simple_preprocess_df(
        df_jobs_to_be_processed=df_all,
    )

    # Arrange Categories
    df_all = categorize_df_gender_age(df_all)

    # Gender Recode
    df_all = dummy_code_df_gender_age(df_all)

    print('DF Processed:')
    df_all.info()
    df_all['Gender'].value_counts()
    df_all['Age'].value_counts()
    df_gender_age_info(df_all)
    df_all['English Requirement'].value_counts()
    df_all['Dutch Requirement'].value_counts()

    df_all.to_pickle(f'{df_dir}df_processed_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format}')


    df_all.to_csv(f'{df_dir}df_processed_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format_backup}',
        index=False)

elif classification_final_from_df_jobs_list is False:
    df_all = pd.read_pickle(f'{df_dir}df_processed_age_limit-{age_limit}_age_ratio-{age_ratio}_gender_ratio-{gender_ratio}.{file_save_format}')
    print('DF Processed:')
    df_all.info()
    df_all['Gender'].value_counts()
    df_all['Age'].value_counts()
    df_gender_age_info(df_all)
    df_all['English Requirement'].value_counts()
    df_all['Dutch Requirement'].value_counts()


DF Processed:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21204 entries, 0 to 21203
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Search Keyword       21204 non-null  object        
 1   Platform             21204 non-null  object        
 2   Job ID               21204 non-null  object        
 3   Job Title            21204 non-null  object        
 4   Company Name         21201 non-null  object        
 5   Location             21204 non-null  object        
 6   Job Description      21204 non-null  object        
 7   Rating               9226 non-null   float64       
 8   Employment Type      20563 non-null  object        
 9   Company URL          19907 non-null  object        
 10  Job URL              21204 non-null  object        
 11  Job Age              21204 non-null  object        
 12  Job Age Number       21204 non-null  object        
 13  Collection Date  

Mixed Gender    16134
Male             3334
Female           1736
Name: Gender, dtype: int64

Mixed Age         15503
Older Worker       3898
Younger     1803
Name: Age, dtype: int64


DF INFO:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21204 entries, 0 to 21203
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Search Keyword       21204 non-null  object        
 1   Platform             21204 non-null  object        
 2   Job ID               21204 non-null  object        
 3   Job Title            21204 non-null  object        
 4   Company Name         21201 non-null  object        
 5   Location             21204 non-null  object        
 6   Job Description      21204 non-null  object        
 7   Rating               9226 non-null   float64       
 8   Employment Type      20563 non-null  object        
 9   Company URL          19907 non-null  object        
 10  Job URL              21204 non-null  object        
 11  Job Age              21204 non-null  object        
 12  Job Age Number       21204 non-null  object        
 13  Collection Date     

No     20192
Yes     1012
Name: English Requirement, dtype: int64

No     20753
Yes      451
Name: Dutch Requirement, dtype: int64